<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/channel_steepness_and_concavity_examples/extracting_knickpoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting knickpoints

Last updated by Simon M. Mudd on 22/04/2024

`lsdtt-chi-mapping` is one of the command line tools included in *LSDTopoTools*. This part of *LSDTopoTools* contains many routines for looking at channel steepness indices as well as concavity indices. In this example we will extract some knickpoints.

If you want more background you can read this papers:

* Gailleton, B., Mudd, S. M., Clubb, F. J., Peifer, D., and Hurst, M. D.: A segmentation approach for the reproducible extraction and quantification of knickpoints from river long profiles, Earth Surf. Dynam., 7, 211–230, https://doi.org/10.5194/esurf-7-211-2019, 2019.

If you use this software for your research please cite the above paper as appropriate and this software: https://doi.org/10.5281/zenodo.3245040

## Stuff we need to do if you are in colab (not required in the lsdtopotools pytools container)

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this.
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Download some data** section.

First we install `lsdtopotools`. The first line downloads the package and the second installs it. The `/dev/null` stuff is just to stop the notebook printing a bunch of text to screen.  

In [ ]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.9.

In [ ]:
!lsdtt-basic-metrics -v

Now we install `lsdviztools`. We need to test if this is version 0.4.12:

In [ ]:
!pip install lsdviztools  &> /dev/null
import lsdviztools
lsdviztools.__version__


## Download some data

We need to get some data to download.

For this example we will work near Santa Cruz, CA

We are going to download data using the opentopography scraper that is included with `lsdviztools`. You will need to get an opentopography.org account and copy in your API key.

You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "SC"
source_name = "COP30"

SC_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [36.93516810706785, -122.33619592430614],
                        upper_right_coordinates = [37.231287577974676, -122.02562650945539],
                        prefix = Dataset_prefix,
                        api_key_file = your_OT_api_key_file)
SC_DEM.print_parameters()
SC_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

We can check to see if the file has downloaded

In [ ]:
!ls

## A simple hillshade image

We can run `lsdtt-chi-mapping` with a little interface in `lsdviztools` called the `lsdtt_driver`. Here we are only going to write a hillshade raster.  

First we check if we are on version 0.4.12 or above.

In [ ]:
import lsdviztools
lsdviztools.__version__

I we are in the correct version, carry on the the next steps.

In [ ]:
import lsdviztools.lsdmapwrappers as lsdmw

In [ ]:
Dataset_prefix = "SC"
source_name = "COP30"

lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

We can plot the image using a `lsdviztools` function. I am printing this to a file and then using the image display option since it allows us to make the figure a bit bigger and in addition you can keep the png image for other uses later.

You can save the figure as well. This allows you to show it in this notebook in a larger format:

In [ ]:
%%capture
DataDirectory = "./"
# Use the capture comment to get rid of all the text
# But it will also not display an inline image
# So you need to call the image from the next line of code
# WARNING %%capture must be the first line in a code block (you can't even have comments above)
Base_file = r_prefix
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth",
                                 save_fig=True, size_format="geomorphology",dpi=600)

In [ ]:
print(this_img)
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

## Choosing basin outlets

One way to get all the basins we want it to make a file with the latitude and longitude of the outlets. We can go into google maps and right click on locations, and then copy these into a latitude and longitude list.

In [ ]:
# Import pandas library
import pandas as pd

data = [ [37.103977560491835, -122.27688719084775],
         [37.04698035269795, -122.22620569548667],
         [37.01252435334153, -122.19055138486662],
         [36.98739273623374, -122.1518657359513],
         [36.98116618978382, -122.1381525397069]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['latitude', 'longitude'])

df.to_csv("basin_outlets.csv",index=False)
df.head()

In [ ]:
import rasterio
raster = rasterio.open("SC_COP30_UTM_hs.bil")
# Get the CRS
crs = raster.crs

print(crs)

In [ ]:
# Make a geopandas dataframe from the basin outlets
import geopandas as gpd
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude,df.latitude), crs="EPSG:4326")
gdf = gdf.to_crs(raster.crs)
gdf.head(5)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

bbox = raster.bounds
bbox_list = [bbox.left, bbox.right, bbox.bottom, bbox.top]


# Create a figure and axes
fig, ax = plt.subplots()

img=raster.read(1)
img = np.where(img == -9999, np.nan, img)

# Add the raster image to the axes
# Note: you might need to adjust the extent and/or coordinate system to match your data
ax.imshow(img, extent=bbox_list, cmap='gray')

gdf.plot(ax=ax, markersize=5, color='red')
plt.clf

In [ ]:
lsdtt_parameters = {"print_basin_raster" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()


Now lets look at the channels and basins using `lsdviztools`.
Again, `lsdviztools` does not make interactive plots and is much less flexible than `folium` or `imshow` but it is optimised for publication-ready figures.

In [ ]:
!lsdtt-chi-mapping Test_01.driver

## Look at the basins

In [ ]:
%%capture
import lsdviztools.lsdmapwrappers as lsdmw
DataDirectory = "./"
Dataset_prefix = "SC"
source_name = "COP30"
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
Base_file = r_prefix
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth",
                             size_format="geomorphology",dpi=600, save_fig = True)

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

## Have a look at the $k_{sn}$ values

In [ ]:
lsdtt_parameters = {"m_over_n" : "0.45",
                    "print_segmented_M_chi_map_to_csv" : "true",
                    "get_basins_from_outlets" : "true",
                    "basin_outlet_csv" : "basin_outlets.csv"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

You could plot the points over a hillshade this way:

In [ ]:
#%%capture
points_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file,cmap="viridis",
                                            points_fname = "SC_COP30_UTM_MChiSegmented.csv",
                                            size_format="geomorphology",dpi=600,
                                            save_fig = True,
                                            column_for_plotting = "m_chi",
                                            column_for_scaling = "drainage_area",
                                            scaled_data_in_log = True,
                                            max_point_size = 14, min_point_size =5)

In [ ]:
from IPython.display import display, Image
display(Image(filename="./SC_COP30_UTM_channels_with_basins.png", width=800))

Or you could have a fancier plot:

In [ ]:
ksn_img = lsdmw.PrintChiChannelsAndBasins(DataDirectory,Base_file,
                                          ChannelFileName = "SC_COP30_UTM_MChiSegmented.csv",
                                          add_basin_labels = False, cmap = "viridis",
                                          cbar_loc = "right",
                                          size_format="geomorphology",dpi=600,
                                          fig_format = "png",
                                          plotting_column="m_chi",
                                          colorbarlabel = "$\mathrm{log}_{10} \; \mathrm{of} \; k_{sn}$",
                                          out_fname_prefix = "ksn_plot")

In [ ]:
!ls

In [ ]:
from IPython.display import display, Image
display(Image(filename="./ksn_plot_chi_channels_and_basins.png", width=800))

## Now get the knickpoints

In [ ]:
lsdtt_parameters = {"m_over_n" : "0.45",
                    "ksn_knickpoint_analysis" : "true"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-chi-mapping",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!ls

Lets plot some knickpoints. In this case many of the knickpoints are distributed, so we probably would play a bit with the settings to get a better result.

See here:
https://lsdtopotools.github.io/LSDTT_documentation/LSDTT_knickpoint_analysis.html#_the_methodology

In [ ]:
#%%capture
points_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file,cmap="viridis",
                                            points_fname = "SC_COP30_UTM_ksnkp.csv",
                                            size_format="geomorphology",dpi=600,
                                            save_fig = True,
                                            column_for_plotting = "delta_ksn",
                                            column_for_scaling = "drainage_area",
                                            scaled_data_in_log = True,
                                            max_point_size = 14, min_point_size =5)

In [ ]:
from IPython.display import display, Image
display(Image(filename=points_img, width=800))